In [1]:
import pandas as pd

In [ ]:
path = r'C:\Users\barka\OneDrive\Final Project\Data\Manhattan\2019\Model Learning'
less_than_10_df = pd.read_csv(path + '')
beteween_10_and_15_df = pd.read_csv()
more_than_15_df = pd.read_csv()